# **Imports the needed Libaries and the File (podcast_id_titles.csv)**

In [84]:
# Imports the functions needed for the scrapping
from googlesearch import search
import urllib
import webbrowser 
from bs4 import BeautifulSoup as BS
import io
import csv
import pandas as pd
import requests
import re

In [85]:
# Imports the podcast_id_titles from your local directory
from google.colab import files
uploaded = files.upload()

Saving podcast_id_titles.csv to podcast_id_titles (3).csv


In [86]:
#  Converts the file to a df then creates a list with the titles
df = pd.read_csv('podcast_id_titles.csv', delimiter=',')
query_list = list(df['title'])
query_list

['Sacred Learning',
 'Behind the Rind: The Story & Science of Cheese',
 'Famous Failures',
 'Business of Home Podcast',
 'Her Story of Success',
 'Shake Up Learning Show',
 'Kicking it with Kris and Karlos',
 'In the Spirit of Horse',
 'Grim Grinning Hosts',
 'Cauldron - A History Of The World Battle By Battle',
 'Attention Horrors',
 'Everyday Sublime',
 'PopHealth Podcast',
 'Business with Purpose',
 "Mass' World",
 'Gaylords of Darkness',
 'Chronicles On Tape',
 'Inspired Money',
 'Random Movement Podcast',
 'Earth Oddity',
 'You, Me, Empathy: Sharing Our Mental Health Stories',
 'LIFE TALKS with Stephen and Pam',
 'Backstage: Chargers',
 'Airplane Owner Maintenance - By Dean Showalter',
 'Rebel & Muse',
 'Dare To Be Legendary with Blake Pinsker',
 'Ace Weekly',
 'Mind Your Fitness Podcast by Ali Lampert',
 'Off Book: The Black Theatre Podcast',
 'Chez Fondilez',
 'Schooled Ya!',
 'Sail On: The Beach Boys Podcast',
 'ADHD for Smart Ass Women with Tracy Otsuka',
 'Under the Hood',
 '

# **DUCK DUCK GO SEARCH METHOD**
ULTAMATELY NOT USED IN PROJECT BUT SAVED FOR FUTURE USE

*   Usually only gets 10-20 Links at a time.
*   Not as realiable as Google Search Method


 




In [ ]:
# Duck Duck Go Search and returns the URL for the podcast addict page with the name of the podcast
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:84.0) Gecko/20100101 Firefox/84.0",
}

duck_addict_list = []

for name in query_list:
    print(name)
    # Does String Minipulation to get rid of spaces and weird characters
    if name[0] == ' ': 
      name = name[1:]
    elif name[-1] == ' ': 
      name = name[:-1]
    name = name.replace(" ","+")
    name = name.replace("'","")
    name = name.replace(",","")
    name = name.replace("–","")
    name = name.replace(":","")
    name = name.replace("™","")
    name = name.replace("!","")
    name = name.replace("(","")
    name = name.replace(")","")
    name = name.replace("&","and")
    name = name.replace("+++","+")
    name = name.replace("++","+")

    # Creates the URL that can be used in searching
    url = "https://duckduckgo.com/html/?q=podcast+addict+"+name
    page = requests.get(url, headers=headers).text
    soup = BS(page, 'html.parser').find_all("a", class_="result__url")
    i = 0
    for link in soup:
      if i < 1: 
        duck_addict_list.append(link['href'])
        break
# Prints the list of URLS found
print(duck_addict_list)

Sacred Learning
Behind the Rind: The Story & Science of Cheese
Famous Failures
Business of Home Podcast
Her Story of Success
Shake Up Learning Show
Kicking it with Kris and Karlos
In the Spirit of Horse
Grim Grinning Hosts
Cauldron - A History Of The World Battle By Battle
Attention Horrors
Everyday Sublime
PopHealth Podcast
Business with Purpose
Mass' World
Gaylords of Darkness
Chronicles On Tape
Inspired Money
Random Movement Podcast
Earth Oddity
You, Me, Empathy: Sharing Our Mental Health Stories
LIFE TALKS with Stephen and Pam
Backstage: Chargers
Airplane Owner Maintenance - By Dean Showalter
Rebel & Muse
Dare To Be Legendary with Blake Pinsker
Ace Weekly
Mind Your Fitness Podcast by Ali Lampert
Off Book: The Black Theatre Podcast
Chez Fondilez
Schooled Ya!
Sail On: The Beach Boys Podcast
ADHD for Smart Ass Women with Tracy Otsuka
Under the Hood
LifePoint Church Podcast
One Billion
REBEL Cast
Flip Talk Rookie Playbook
Hilliard Guess' Screenwriters Rant Room
Wellness Revolutionaries

In [ ]:
# Prints the list of URLS found
print(duck_addict_list)
# Prints the Length of the URLS fouund
print(len(duck_addict_list))


['https://podcastaddict.com/podcast/sacred-learning/474437', 'https://podcastaddict.com/podcast/behind-the-rind-the-story-science-of-cheese/2161191', 'https://podcastaddict.com/podcast/famous-failures/2935592', 'https://podcastaddict.com/podcast/2484225', 'https://podcastaddict.com/podcast/2150981', 'https://podcastaddict.com/podcast/shake-up-learning-show/2346009', 'https://podcastaddict.com/podcast/2306804', 'https://podcastaddict.com/podcast/2380906', 'https://podcastaddict.com/podcast/grim-grinning-hosts/2158658', 'https://podcastaddict.com/podcast/2328768', 'https://www.whatsdannydoing.com/blog/short-inspirational-stories-with-a-moral', 'https://podcastaddict.com/podcast/taboo-fx/2380388', 'https://podcastaddict.com/podcast/generation-anthropocene/4033529', 'https://podcastaddict.com/podcast/wholesale-made-easy/2277921', 'https://podcastaddict.com/podcast/3784839', 'https://podcastaddict.com/podcast/1733328', 'https://podcasts.apple.com/us/podcast/sellercast-sell-more-on-amazon/id

In [ ]:
# This code is meant to do some error checking to get rid of links that went to specfic episodes instead of the the actual podcast page
for url in duck_addict_list:
    if (url[0:34] != 'https://podcastaddict.com/podcast/'):
      duck_addict_list.remove(url)

print(duck_addict_list)
print(len(duck_addict_list))

['https://podcastaddict.com/podcast/sacred-learning/474437', 'https://podcastaddict.com/podcast/behind-the-rind-the-story-science-of-cheese/2161191', 'https://podcastaddict.com/podcast/famous-failures/2935592', 'https://podcastaddict.com/podcast/2150981', 'https://podcastaddict.com/podcast/shake-up-learning-show/2346009', 'https://podcastaddict.com/podcast/2380906', 'https://podcastaddict.com/podcast/grim-grinning-hosts/2158658', 'https://podcastaddict.com/podcast/2328768', 'https://podcastaddict.com/podcast/taboo-fx/2380388', 'https://podcastaddict.com/podcast/generation-anthropocene/4033529', 'https://podcastaddict.com/podcast/wholesale-made-easy/2277921', 'https://podcastaddict.com/podcast/3784839', 'https://podcastaddict.com/podcast/1733328', 'https://podcastaddict.com/podcast/2278860', 'https://podcastaddict.com/podcast/faith-in-business/2235932', 'https://podcastaddict.com/podcast/rhee-gold-s-dancelife/2548925']
16


# **GOOGLE SEARCH METHOD**
Returns about 40 Links. Can run every 10-15 Min. Also can take a couple minutes to run based on WiFi Speeds. If you get a 429 Error that just means that it exceed the number of searches per a batch and stopped

In [87]:
# Does a Google Search and returns the URL for the podcast addict page with the name of the podcast
google_addict_list = []
for name in query_list:
    for result in search(name,  tld='com', lang='en', domains = ["podcastaddict.com/podcast"], num=2, start=0, stop=1, pause=3.0):
        google_addict_list.append(result)

print(google_addict_list)

HTTPError: ignored

In [88]:
print(google_addict_list)
print(len(google_addict_list))

['https://podcastaddict.com/podcast/3756105', 'https://podcastaddict.com/podcast/2440156', 'https://podcastaddict.com/podcast/2214856', 'https://podcastaddict.com/podcast/2484225', 'https://podcastaddict.com/podcast/her-story-of-success/2150981', 'https://www.podcastaddict.com/podcast/2302923', 'https://podcastaddict.com/podcast/2306804', 'https://podcastaddict.com/podcast/3219279', 'https://podcastaddict.com/podcast/2537203', 'https://podcastaddict.com/podcast/2101938', 'https://podcastaddict.com/podcast/3019915', 'https://podcastaddict.com/podcast/3041916', 'https://podcastaddict.com/podcast/2138474', 'https://podcastaddict.com/podcast/1861604', 'https://podcastaddict.com/podcast/3231758', 'https://podcastaddict.com/podcast/2254406', 'https://podcastaddict.com/podcast/2918625', 'https://www.podcastaddict.com/podcast/3344106', 'https://podcastaddict.com/podcast/1594679', 'https://podcastaddict.com/podcast/2219815', 'https://podcastaddict.com/podcast/2532027', 'https://podcastaddict.co

# **PODCAST ADDICT SCRAPING**
Using the Links that we got from Duck Duck Go or the Google Search method we will now scrap the link that directly goes to the RSS for each individual podcast from podcastaddicts.com

THIS SITE IS UNREALIABLE AND CONSTANTLY CHANGING... IF YOU ENCOUNTER ANY ISSUES PLEASE REACH OUT. Proof can be provided that this script worked. 

In [89]:
# Needed to open URL from podcastaddict.com
header= {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
    'AppleWebKit/537.11 (KHTML, like Gecko) '
    'Chrome/23.0.1271.64 Safari/537.11',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
    'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
    'Accept-Encoding': 'none',
    'Accept-Language': 'en-US,en;q=0.8',
    'Connection': 'keep-alive'}

In [90]:
# Function that opens the podcast addict link and returns the link associated with the RSS feed
def get_rss(url): 
    # Makes the request and opens the addict link
    req = urllib.request.Request(url, headers=header) 
    page = urllib.request.urlopen(req).read()
    # Parses out the URL and returns the link to the RSS
    soup = BS(page, "html.parser")
    i = 0
    for link in soup.find_all('a'):
        if i == 30: 
          rss = link.get('href')
          print(rss)
          return rss
        else: 
          i += 1
  

If you get a 404 Error that just means that the script is being blocked from Podcast Addict for to many requests. Try again in a bit

In [92]:
# Function that iterates through the Podcast Addict links and returns the RSS links for each podcast
#duck_RSS_links = []
google_RSS_links = []

#for url in duck_addict_list: 
  #result = get_rss(url)
  #duck_RSS_links.append(result)

for url in google_addict_list: 
  result = get_rss(url)
  google_RSS_links.append(result)

#print(duck_RSS_links)
print(google_RSS_links)
print(len(google_RSS_links))

HTTPError: ignored

In [ ]:
# This Downloads the CSV to the desktop
df = pd.DataFrame(google_RSS_links)
df.to_csv('RSS_Links.csv', encoding = 'utf-8-sig') 
files.download('RSS_Links.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>